Coastal Mountain Walks

In [ ]:
import polars as pl

columns = ['Start time', 'End time', 'Move Minutes count', 'Calories (kcal)', 'Distance (m)', 'Heart Points']

all_data = []
time_format = '%H:%M:%S.%3f%z'

for day in range(2, 17):
    data = pl.read_csv(f'/home/rg/data/0_review/Google-Fit/Daily activity metrics/2026-01-{day:02d}.csv')
    # Sample Start Time 00:00:00.000+01:00
    subset = data.select(pl.col(columns)).with_columns(
        pl.lit(day).alias('Day'),
        pl.col('Start time').str.strptime(pl.Time, format=time_format).alias('Start time'),
        pl.col('End time').str.strptime(pl.Time, format=time_format).alias('End time'),
        pl.col('Move Minutes count'),
        pl.col('Calories (kcal)'),
        pl.col('Distance (m)'),
        (pl.col('Distance (m)') / (pl.col('Move Minutes count') * 60)).alias('Avg Speed (m/s)'),
    )
    all_data.append(subset)

df = pl.concat(all_data)

In [174]:
df.head(10)

Start time,End time,Move Minutes count,Calories (kcal),Distance (m),Heart Points,Speed (m/s),Day
time,time,i64,f64,f64,f64,f64,i32
00:00:00,00:15:00,null,16.427083,null,null,null,2
00:15:00,00:30:00,null,16.427083,null,null,null,2
00:30:00,00:45:00,null,16.427083,null,null,null,2
00:45:00,01:00:00,null,16.427083,null,null,null,2
01:00:00,01:15:00,null,16.427083,null,null,null,2
01:15:00,01:30:00,null,16.427083,null,null,null,2
01:30:00,01:45:00,null,16.427083,10.665088,null,null,2
01:45:00,02:00:00,null,16.427083,null,null,null,2
02:00:00,02:15:00,1,16.427083,25.8277,null,0.430462,2


In [162]:
df.group_by('day').agg(pl.all().exclude(['Start time', 'End time']).sum())

day,Move Minutes count,Calories (kcal),Distance (m),Heart Points,start_time,end_time,avg_speed_ms
i32,i64,f64,f64,f64,time,time,f64
2,235,2239.760015,11408.168131,42.0,null,null,24.352045
3,312,2436.302598,19900.949137,144.0,null,null,30.51104
4,224,2242.17852,11800.489428,53.0,null,null,21.119241
5,309,2578.252809,20437.568658,163.0,null,null,33.770268
6,303,2414.362574,16580.448128,115.0,null,null,25.696882
…,…,…,…,…,…,…,…
12,280,2415.356634,15498.938255,75.0,null,null,25.649736
13,407,2791.036614,22704.830415,109.0,null,null,32.847006
14,276,2440.606566,15157.917143,75.0,null,null,24.905675
